# Find the Equivalent Ellipse

This notebook makes defines an object class which takes cutout shape defining points as an input along with a user defined cutout factor `circle_fit` function and returns the equivalent ellipse. For information on how to define the `circle_fit` function see the [example given in SectorIntegration.ipynb](SectorIntegration.ipynb#Example).

To calculate the equivalent ellipse a seriese of objects are used which can each be found in their own notebook. These are outlined as such:

 * Finding the position of maximum dose output defined in [FindingCentre.ipynb](FindingCentre.ipynb)
   * Which makes use of [SectorIntegration.ipynb](SectorIntegration.ipynb)
 * Cutout straightening defined in [CutoutStraightening.ipynb](CutoutStraightening.ipynb)
 * Ellipse fitting found in [EllipseFitting.ipynb](EllipseFitting.ipynb)
 

In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import pylab
%matplotlib inline

import os, sys
sys.path.append(os.path.abspath('..'))

import nbimport
sys.meta_path.append(nbimport.NotebookFinder())

from EquivalentEllipse.FindingCentre import find_centre
from EquivalentEllipse.CutoutStraightening import straighten_cutout
from EquivalentEllipse.EllipseFitting import fit_ellipse

importing IPython notebook from /home/simonbiggs/github/equivalent-ellipse-spline-modelling/program/EquivalentEllipse/FindingCentre.ipynb
importing IPython notebook from /home/simonbiggs/github/equivalent-ellipse-spline-modelling/program/EquivalentEllipse/SectorIntegration.ipynb
importing IPython notebook from /home/simonbiggs/github/equivalent-ellipse-spline-modelling/program/EquivalentEllipse/CutoutStraightening.ipynb
importing IPython notebook from /home/simonbiggs/github/equivalent-ellipse-spline-modelling/program/EquivalentEllipse/EllipseFitting.ipynb


/usr/local/lib/python3.4/dist-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
